In [37]:
#16 Agosto 2024
#Juego de naves
#Jamith Bolaños Vidal
#Prueba ingreso Kotland

import pygame
import random

# Inicializar PyGame
pygame.init()

# Configuraciones de pantalla
ANCHO, ALTO = 800, 600
pantalla = pygame.display.set_mode((ANCHO, ALTO))
pygame.display.set_caption("Juego de Nave")

# Colores
NEGRO = (0, 0, 0)
BLANCO = (255, 255, 255)
ROJO = (255, 0, 0)

# Reloj
reloj = pygame.time.Clock()
FPS = 60

# Cargar imágenes
imagen_nave = pygame.image.load("assets/nave.png").convert_alpha()
imagen_roca = pygame.image.load("assets/roca.png").convert_alpha()

# Inicializar variables
vidas = 4
puntaje = 0

class Nave(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = imagen_nave  # Usar la imagen cargada
        self.rect = self.image.get_rect()
        self.rect.centerx = ANCHO // 2
        self.rect.bottom = ALTO - 10

    def update(self):
        teclas = pygame.key.get_pressed()
        if teclas[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x -= 5
        if teclas[pygame.K_RIGHT] and self.rect.right < ANCHO:
            self.rect.x += 5

    def disparar(self):
        bala = Bala(self.rect.centerx, self.rect.top)
        todas_las_sprites.add(bala)
        balas.add(bala)

class Roca(pygame.sprite.Sprite):
    def __init__(self, velocidad_min, velocidad_max):
        super().__init__()
        self.image = imagen_roca  # Usar la imagen cargada
        self.rect = self.image.get_rect()
        self.rect.x = random.randint(0, ANCHO - self.rect.width)
        self.rect.y = random.randint(-100, -40)
        self.velocidad_y = random.randint(velocidad_min, velocidad_max)

    def update(self):
        self.rect.y += self.velocidad_y
        if self.rect.top > ALTO:
            self.rect.x = random.randint(0, ANCHO - self.rect.width)
            self.rect.y = random.randint(-100, -40)
            self.velocidad_y = random.randint(5, 10)

class Bala(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((5, 10))
        self.image.fill(ROJO)
        self.rect = self.image.get_rect()
        self.rect.centerx = x
        self.rect.bottom = y

    def update(self):
        self.rect.y -= 10
        if self.rect.bottom < 0:
            self.kill()

# Grupos de sprites
todas_las_sprites = pygame.sprite.Group()
balas = pygame.sprite.Group()
rocas = pygame.sprite.Group()

# Crear nave
nave = Nave()
todas_las_sprites.add(nave)



def mostrar_texto(superficie, texto, tamaño, color, x, y):
    fuente = pygame.font.Font(None, tamaño)
    texto_superficie = fuente.render(texto, True, color)
    texto_rect = texto_superficie.get_rect()
    texto_rect.center = (x, y)
    superficie.blit(texto_superficie, texto_rect)

def bucle_juego(nivel):
    global vidas, puntaje
    corriendo = True
    # Ajustar la dificultad según el nivel
    if nivel == 1:
        velocidad_min = 5
        velocidad_max = 10
        num_rocas = 5
    elif nivel == 2:
        velocidad_min = 10
        velocidad_max = 15
        num_rocas = 10
    elif nivel == 3:
        velocidad_min = 15
        velocidad_max = 20
        num_rocas = 12
    # Crear rocas
    for i in range(num_rocas):
        roca = Roca(velocidad_max, velocidad_max)
        todas_las_sprites.add(roca)
        rocas.add(roca)
    
    while corriendo:
        reloj.tick(FPS)

        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False
                pygame.quit()
                return puntuacion
            elif evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_SPACE:
                    nave.disparar()

        # Actualizar sprites
        todas_las_sprites.update()

        # Detectar colisiones entre rocas y la nave
        colisiones = pygame.sprite.spritecollide(nave, rocas, False)
        for colision in colisiones:
            vidas -= 1  # Reducir una vida por cada colisión
            if vidas <= 0:
                guardar_puntuacion(puntaje)
                vidas = 4 # se reinician las vida sy el puntaje
                puntaje = 0
                corriendo = False  # Termina el juego si se quedan sin vidas
                mostrar_game_over()  # Mostramos el mensaje de "Game Over"
                #menu_principal()  # Volvemos al menú principal
                
                return  # Salimos de la función para evitar que siga ejecutándose
            # Resetear la posición de la roca que colisionó
            colision.rect.y = random.randint(-100, -40)
            colision.rect.x = random.randint(0, ANCHO - colision.rect.width)

        # Detectar colisiones entre balas y rocas
        balas_rocas_colision = pygame.sprite.groupcollide(balas, rocas, True, True)
        for colision in balas_rocas_colision:
            puntaje += 3  # Sumar puntos por cada roca destruida
            # Crear una nueva roca
            nueva_roca = Roca(5, 10)
            todas_las_sprites.add(nueva_roca)
            rocas.add(nueva_roca)

        # Dibujar todo
        pantalla.fill(NEGRO)
        todas_las_sprites.draw(pantalla)
        mostrar_texto(pantalla, f"Vidas: {vidas}", 24, BLANCO, 70, 10)
        mostrar_texto(pantalla, f"Puntaje: {puntaje}", 24, BLANCO, 730, 10)
        pygame.display.flip()
        
def guardar_puntuacion(puntuacion):
    with open("puntuaciones.txt", "a") as archivo:
        archivo.write(f"{puntuacion}\n")

def seleccionar_nivel():
    corriendo = True
    nivel = 1
    while corriendo:
        pantalla.fill(NEGRO)
        mostrar_texto(pantalla, "Selecciona Nivel (1-3)", 74, BLANCO, 390, 150)
        mostrar_texto(pantalla, f'Nivel seleccionado: {nivel}', 36, BLANCO, 390, 300)

        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False
                pygame.quit()
                return None
            if evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_1:
                    nivel = 1
                elif evento.key == pygame.K_2:
                    nivel = 2
                elif evento.key == pygame.K_3:
                    nivel = 3
                elif evento.key == pygame.K_RETURN:
                    corriendo = False

        pygame.display.flip()

    return nivel

def mostrar_game_over():
    pantalla.fill(NEGRO)
    mostrar_texto(pantalla, "GAME OVER", 74, ROJO, ANCHO // 2, ALTO // 3)
    mostrar_texto(pantalla, "Presiona ENTER para salir", 36, BLANCO, ANCHO // 2, ALTO // 2)
    pygame.display.flip()
    esperando = True
    while esperando:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                pygame.quit()
                exit()
            if evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_RETURN:
                    esperando = False

def menu_principal():
    corriendo = True
    while corriendo:
        pantalla.fill(NEGRO)
        mostrar_texto(pantalla, "Juego de Nave", 74, BLANCO, 400, 150)
        mostrar_texto(pantalla, "Presiona ENTER para jugar", 36, BLANCO, 390, 300)
        mostrar_texto(pantalla, "Presiona ESC para salir", 36, BLANCO, 390, 350)

        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                #corriendo = False
                pygame.quit()
                return
            if evento.type == pygame.KEYDOWN:
                if evento.key == pygame.K_RETURN:
                    nivel = seleccionar_nivel()
                    if nivel is None:  # Si la ventana se cierra en el menú de selección de nivel
                        return
                    puntuacion = bucle_juego(nivel)
                    corriendo = False
                if evento.key == pygame.K_ESCAPE:
                    corriendo = False

        pygame.display.flip()        
        

# Llamada para iniciar el juego
menu_principal()

pygame.quit()
